In [329]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [330]:
using QuantumOptics

In [331]:
function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [332]:
Nx = 5; Ny = 5; N=Nx*Ny; q = 5; N_cut = 5
PN = 2
U = 2

2

In [333]:
using LinearAlgebra

basis = NLevelBasis(N)

NLevel(N=25)

In [334]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=25x25)
  basis: NLevel(N=25)sparse([2, 5, 6, 21, 1, 3, 7, 22, 2, 4  …  22, 24, 4, 19, 23, 25, 5, 20, 21, 24], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  23, 23, 24, 24, 24, 24, 25, 25, 25, 25], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -0.30901699437494745 - 0.9510565162951536im, -0.30901699437494745 + 0.9510565162951536im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 0.8090169943749476 + 0.587785252292473im, 0.8090169943749476 - 0.587785252292473im, -1.0 + 0.0im, -1.0 + 0.0im, -0.30901699437494723 + 0.9510565162951536im, -0.30901699437494723 - 0.9510565162951536im, -1.0 + 0.0im, -1.0 + 0.0im], 25, 25)

In [335]:
eigenenergies(dense(H1))

25-element Vector{Float64}:
 -2.966447989143374
 -2.966447989143373
 -2.966447989143372
 -2.966447989143372
 -2.9664479891433717
 -0.9021130325903087
 -0.9021130325903075
 -0.902113032590307
 -0.9021130325903065
 -0.9021130325903061
 -0.3819660112501061
 -0.381966011250105
 -0.3819660112501049
 -0.38196601125010354
 -0.38196601125010315
  1.3484140003934757
  1.3484140003934764
  1.348414000393477
  1.3484140003934773
  1.3484140003934784
  2.9021130325903055
  2.9021130325903055
  2.902113032590307
  2.9021130325903077
  2.902113032590309

In [336]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [337]:
function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [338]:
sub_states = get_sub_states(H1, N_cut);

In [339]:
function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [340]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [341]:
function Subspace_Op(SP_Op, P)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [342]:
H1_sub = Subspace_Op(H1, P)

Operator(dim=5x5)
  basis: Subspace(superbasis=NLevel(N=25), states:5)
 -2.96645-0.0im       3.3e-16+4.0e-17im  …  -1.0e-17+0.0im
  3.3e-16+1.0e-17im  -2.96645-1.1e-16im     -6.0e-17-6.0e-17im
 -6.0e-17-6.0e-17im  -2.8e-16-1.7e-16im          0.0+3.9e-16im
      0.0+3.0e-17im   4.4e-16-2.2e-16im      7.8e-16+2.2e-16im
  1.0e-17-6.0e-17im   6.0e-17+1.1e-16im     -2.96645-1.0e-17im

In [343]:
num_sub_list = []
for m in 1:N
    NM = transition(basis, m, m)
    NMP = Subspace_Op(NM, P)
    push!(num_sub_list, NMP)
end

In [344]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)

In [345]:
function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub)

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)sparse([1, 2, 3, 4, 5, 1, 2, 3, 4, 5  …  11, 12, 13, 14, 15, 5, 9, 12, 14, 15], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  14, 14, 14, 14, 14, 15, 15, 15, 15, 15], ComplexF64[-5.932895978286747 - 0.0im, 4.7e-16 + 1.0e-17im, -8.0e-17 - 8.0e-17im, 0.0 + 4.0e-17im, 2.0e-17 - 8.0e-17im, 4.7e-16 + 6.0e-17im, -5.932895978286745 - 1.1e-16im, -2.8e-16 - 1.7e-16im, 4.4e-16 - 2.2e-16im, 6.0e-17 + 1.1e-16im  …  0.0 + 3.9e-16im, -3.9e-16 - 1.7e-16im, 1.1e-15 + 3.1e-16im, -5.932895978286741 + 1.0e-16im, 1.1e-15 - 1.6e-16im, -2.0e-17 + 0.0im, -8.0e-17 - 8.0e-17im, 0.0 + 5.5e-16im, 1.1e-15 + 3.1e-16im, -5.932895978286742 - 3.0e-17im], 15, 15)

In [346]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [347]:
num_mb_list = []
for m in 1:N
    NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
    push!(num_mb_list, NMP)
end

In [348]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U)

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  15, 15, 15, 15, 15, 15, 15, 15, 15, 15], ComplexF64[-1.3961619523913684 + 0.0im, 0.00516089848643062 + 9.480078462494e-5im, -0.00177895505230145 + 0.00049843727453398im, 0.00399282885501431 - 0.00143453247944256im, -0.00131503021949532 + 0.00020288669648142im, 0.02792281688785346 - 0.00070959034773484im, 0.00909055521727068 - 0.01535403139591273im, -0.0075359513084783 - 0.00076588183127313im, 0.00472079056687338 + 0.00656687516252759im, -0.00977488531032327 - 0.0075647483561838im  …  0.01449206257970099 + 0.02003977504232739im, -0.01778661214785522 - 0.0244790911354106im, 0.0015192104743605 + 0.0201498139739635im, 0.01547900405689002 + 0.00384444214222339im, 0.00944892721253029 + 0.00680037862784733im, -0.00734972541872334 - 0.00637475057141834im, 0.03935885946773131 + 0

In [349]:
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)
    0.203838+0.0im          …  0.000451391-0.00255317im
   0.0051609+9.48008e-5im      -0.00575143+0.017804im
 -0.00177896+0.000498437im      0.00172758+0.00287198im
  0.00399283-0.00143453im      -0.00259561+0.00782752im
 -0.00131503+0.000202887im      0.00514083+0.00806122im
   0.0279228-0.00070959im   …    0.0144921+0.0200398im
  0.00909056-0.015354im         -0.0177866-0.0244791im
 -0.00753595-0.000765882im      0.00151921+0.0201498im
  0.00472079+0.00656688im         0.015479+0.00384444im
 -0.00977489-0.00756475im       0.00944893+0.00680038im
 -0.00554273+0.0124697im    …  -0.00734973-0.00637475im
  0.00772461-0.00129291im        0.0393589+0.0031757im
   0.0141431-0.0103721im        -0.0245087+0.0169029im
  0.00416563-0.00584565im        0.0157829+0.019494im
 0.000451391+0.00255317im         0.167428+0.0im

In [350]:
H_MB = H1_MB + H_Int_MB
H_mb = H1_MB + Vint_mb

Operator(dim=15x15)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:15)
    -5.72906-0.0im          …  0.000451391-0.00255317im
   0.0051609+9.48008e-5im      -0.00575143+0.017804im
 -0.00177896+0.000498437im      0.00172758+0.00287198im
  0.00399283-0.00143453im      -0.00259561+0.00782752im
 -0.00131503+0.000202887im      0.00514083+0.00806122im
   0.0279228-0.00070959im   …    0.0144921+0.0200398im
  0.00909056-0.015354im         -0.0177866-0.0244791im
 -0.00753595-0.000765882im      0.00151921+0.0201498im
  0.00472079+0.00656688im         0.015479+0.00384444im
 -0.00977489-0.00756475im       0.00944893+0.00680038im
 -0.00554273+0.0124697im    …  -0.00734973-0.00637475im
  0.00772461-0.00129291im        0.0393589+0.0031757im
   0.0141431-0.0103721im        -0.0245087+0.0169029im
  0.00416563-0.00584565im        0.0157829+0.019494im
 0.000451391+0.00255317im         -5.76547-3.0e-17im

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [351]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [352]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-7.5300268624965305, -7.5300268624965305, -7.530026862496529, -7.530026862496527, -7.530026862496525, -7.372090091593086, -7.372090091593082, -7.372090091593081, -7.372090091593078, -7.3720900915930745, -7.296570980770628, -7.296570980770623, -7.296570980770623, -7.29657098077062, -7.296570980770618]
[-5.930026862496534, -5.930026862496532, -5.930026862496527, -5.930026862496525, -5.930026862496525, -5.772090091593088, -5.772090091593086, -5.772090091593082, -5.772090091593081, -5.772090091593078, -5.696570980770627, -5.696570980770625, -5.696570980770623, -5.6965709807706215, -5.696570980770619]
[-5.930812587456487, -5.930812587456478, -5.930812587456469, -5.930812587456454, -5.930812587456437, -5.81785420065733, -5.817854200657318, -5.817854200657314, -5.817854200657309, -5.81785420065728]

In [353]:
E_mb

15-element Vector{Float64}:
 -5.930026862496534
 -5.930026862496532
 -5.930026862496527
 -5.930026862496525
 -5.930026862496525
 -5.772090091593088
 -5.772090091593086
 -5.772090091593082
 -5.772090091593081
 -5.772090091593078
 -5.696570980770627
 -5.696570980770625
 -5.696570980770623
 -5.6965709807706215
 -5.696570980770619